In [1]:
import os
curdir = os.getcwd()
os.chdir("../../")
from PredicTables.impute import ModelImputer
from synthetic_data import generate_synthetic_data_cat
os.chdir(curdir)

In [2]:
import pandas as pd
import numpy as np

In [3]:
X, y = generate_synthetic_data_cat(between_correlation=0.1)
for col in [f"feature_{i}" for i in [10,11,12]]:
    X[col] = X[col].astype(int).astype(str).astype("category")

X = X[[f"feature_{i}" for i in range(10,13)] + [f"feature_{i}" for i in range(10)]]
X.head()

,feature_10,feature_11,feature_12,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,3,3,3,1.316120,0.571763,-0.948255,-0.253405,-0.791555,-1.155067,-0.863983,0.911501,-0.333872,-0.742472
1,3,2,3,0.522886,1.134366,-1.731625,1.131403,-0.161217,-0.936367,0.993590,-0.557153,0.670111,-0.962518
2,2,3,2,-0.871178,-0.025119,-0.033187,-1.469283,-0.660196,0.158048,0.523632,0.135336,-0.427678,-0.592091
3,3,2,3,1.162554,0.572738,0.777561,0.252496,1.518633,-0.136349,-0.054531,-0.334896,0.829051,-0.816178
4,2,2,2,-0.456041,1.064196,1.418257,0.221501,-0.018210,-1.079638,-0.127390,-0.703510,-1.422105,1.164787


In [4]:
Xm = X.copy()
missing_indicator = np.random.binomial(1, 0.2, (Xm.shape[0], 3))
missing_indicator = pd.DataFrame(missing_indicator, columns=['feature_12', 'feature_0', 'feature_1'])

Xm.loc[missing_indicator['feature_12'] == 1, 'feature_12'] = np.nan
Xm.loc[missing_indicator['feature_0'] == 1, 'feature_0'] = np.nan
Xm.loc[missing_indicator['feature_1'] == 1, 'feature_1'] = np.nan
Xm

,feature_10,feature_11,feature_12,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,3,3,3,NaN,0.571763,-0.948255,-0.253405,-0.791555,-1.155067,-0.863983,0.911501,-0.333872,-0.742472
1,3,2,3,NaN,NaN,-1.731625,1.131403,-0.161217,-0.936367,0.993590,-0.557153,0.670111,-0.962518
2,2,3,2,-0.871178,-0.025119,-0.033187,-1.469283,-0.660196,0.158048,0.523632,0.135336,-0.427678,-0.592091
3,3,2,3,1.162554,NaN,0.777561,0.252496,1.518633,-0.136349,-0.054531,-0.334896,0.829051,-0.816178
4,2,2,2,-0.456041,1.064196,1.418257,0.221501,-0.018210,-1.079638,-0.127390,-0.703510,-1.422105,1.164787
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,3,NaN,0.085639,0.206309,0.061159,0.096030,0.703593,0.439108,1.189884,0.668056,0.117579,0.541536
996,1,2,NaN,NaN,NaN,-0.007130,0.574523,-0.727652,0.513568,-1.687766,-0.476256,0.317622,0.077892
997,3,3,3,0.288577,-0.880097,1.442667,1.403458,-1.003496,-0.827813,-1.916010,0.146468,-0.550771,-0.156494
998,2,2,2,-0.464262,-1.421141,0.092551,-0.064225,-0.281926,-0.426088,0.091925,-0.067775,-0.629980,0.671714


In [5]:
missing_mask = pd.DataFrame(missing_indicator == 1, columns=X.columns).fillna(False)
missing_mask.head()

,feature_10,feature_11,feature_12,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,False,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,True,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False


In [7]:
mi = ModelImputer(df=Xm, missing_mask=missing_mask)
mi

models(n=0)

In [8]:
init_impute = initial_impute(Xm)
init_impute.collect().head()

NameError: name 'initial_impute' is not defined

In [ ]:
%run ./src/train_catboost_model.py

In [ ]:
model = train_catboost_model(init_impute.collect().to_pandas(), missing_mask=missing_mask, cv_folds=None)

In [ ]:
iter1 = impute_with_trained_model(init_impute.collect().to_pandas(), missing_mask, model, 0.1)
iter1

,feature_10,feature_11,feature_12,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,3,3,3,1.316120,-0.024111,-0.948255,-0.253405,-0.791555,-1.155067,-0.863983,0.911501,-0.333872,-0.742472
1,3,2,3,0.522886,1.134366,-1.731625,1.131403,-0.161217,-0.936367,0.993590,-0.557153,0.670111,-0.962518
2,2,3,2,-0.871178,-0.030215,-0.033187,-1.469283,-0.660196,0.158048,0.523632,0.135336,-0.427678,-0.592091
3,3,2,3,1.162554,0.572738,0.777561,0.252496,1.518633,-0.136349,-0.054531,-0.334896,0.829051,-0.816178
4,2,2,2,-0.456041,1.064196,1.418257,0.221501,-0.018210,-1.079638,-0.127390,-0.703510,-1.422105,1.164787
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,3,2,0.085639,0.206309,0.061159,0.096030,0.703593,0.439108,1.189884,0.668056,0.117579,0.541536
996,1,2,2,-0.930826,1.496495,-0.007130,0.574523,-0.727652,0.513568,-1.687766,-0.476256,0.317622,0.077892
997,3,3,3,0.288577,-0.880097,1.442667,1.403458,-1.003496,-0.827813,-1.916010,0.146468,-0.550771,-0.156494
998,2,2,2,-0.464262,-1.421141,0.092551,-0.064225,-0.281926,-0.426088,0.091925,-0.067775,-0.629980,0.671714


In [ ]:
original_error = ((np.abs(X.select_dtypes(include=[float, int]) - init_impute.collect().to_pandas().select_dtypes(include=[float, int])))).sum().sum()
original_error / 397

0.8964424134277401

In [ ]:
init_impute.collect().to_pandas().select_dtypes(include=[float, int]) - X.select_dtypes(include=[float, int])

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,0.0,-0.574745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.022137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# init_impute.collect().to_pandas()['feature_0'].dropna().hist()

In [ ]:
# Xm['feature_0'].dropna().hist()

In [ ]:
# iter1['feature_0'].dropna().hist()

In [ ]:
(np.abs(iter1.select_dtypes(include=[float, int]) - X.select_dtypes(include=[float, int])) > np.abs(init_impute.collect().to_pandas().select_dtypes(include=[float, int]) - X.select_dtypes(include=[float, int]))).sum().sum()

119

In [ ]:
def iter_error(df1, df2):
    return ((np.abs(df1.select_dtypes(include=[float, int]) - df2.select_dtypes(include=[float, int]))).sum().sum()) #/ (missing_mask[df1.select_dtypes(include=[float, int]).columns.tolist()].sum().sum())

def pct_error_to_go(df1, df2, df_start):
    return 1 - (iter_error(df1, df2) / iter_error(df_start, df2))

pct_error_to_go(iter1, init_impute.collect().to_pandas(), X)

0.9771831769709324

In [ ]:
missing_mask[X.select_dtypes(include=[float, int]).columns.tolist()].sum().sum()

437.0

In [ ]:
iter_error(iter1, init_impute.collect().to_pandas())

8.120225257463627

In [ ]:
counter = 0
while pct_error_to_go(iter1, init_impute.collect().to_pandas(), X) > 0.1:
    model = train_catboost_model(iter1, missing_mask=missing_mask, cv_folds=None)
    iter1 = impute_with_trained_model(iter1, missing_mask, model, 0.995 ** counter)
    counter += 1
    print(f"Iteration {counter + 1}: Error to go: {pct_error_to_go(iter1, init_impute.collect().to_pandas(), X):.3%}")

iter1

Iteration 2: Error to go: 75.287%
Iteration 3: Error to go: 63.182%
Iteration 4: Error to go: 54.314%
Iteration 5: Error to go: 48.138%
Iteration 6: Error to go: 44.306%
Iteration 7: Error to go: 40.640%
Iteration 8: Error to go: 38.227%
Iteration 9: Error to go: 36.339%
Iteration 10: Error to go: 34.729%
Iteration 11: Error to go: 33.657%
Iteration 12: Error to go: 31.959%
Iteration 13: Error to go: 31.250%
Iteration 14: Error to go: 30.750%
Iteration 15: Error to go: 30.038%
Iteration 16: Error to go: 29.859%
Iteration 17: Error to go: 29.550%
Iteration 18: Error to go: 29.140%
Iteration 19: Error to go: 28.741%
Iteration 20: Error to go: 28.595%
Iteration 21: Error to go: 27.957%
Iteration 22: Error to go: 27.740%
Iteration 23: Error to go: 27.412%
Iteration 24: Error to go: 27.835%
Iteration 25: Error to go: 27.735%
Iteration 26: Error to go: 27.234%
Iteration 27: Error to go: 27.623%
Iteration 28: Error to go: 27.268%
Iteration 29: Error to go: 27.059%
Iteration 30: Error to go: 2

KeyboardInterrupt: 

In [ ]:
model

{'feature_12': [<catboost.core.CatBoostClassifier at 0x2874677d0f0>],
 'feature_0': [<catboost.core.CatBoostRegressor at 0x2874677f4c0>],
 'feature_1': [<catboost.core.CatBoostRegressor at 0x2874677d300>]}

In [ ]:
# import sys
# sys.path.append("../")
# %pip install -q -e ../

In [ ]:
%run ./IterativeRF.py
rf = IterativeRF(X=X_train, debug=True, n_iter=5, rf_hyperparameters="./rf_best_hyperparameters.json")
rf.X

,feature_10,feature_11,feature_12,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
29,2,1,2,-1.220715,-0.682635,0.102570,0.517777,-1.491929,1.662325,-0.691130,NaN,NaN,0.428873
535,3,2,3,0.764024,-1.807970,-0.841080,0.035528,NaN,0.484714,1.496540,-0.632247,NaN,-1.073401
695,2,1,NaN,-0.360238,NaN,-0.282353,-0.426767,NaN,-1.049076,-0.702097,NaN,-1.648665,NaN
557,3,2,2,NaN,-0.211287,-0.267669,-0.065631,NaN,NaN,-0.408133,-0.299202,-0.793206,0.212205
836,1,2,1,-1.640128,NaN,NaN,-1.336151,NaN,-1.051140,-0.065744,-0.723832,-1.419650,-0.081325
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,2,NaN,0.231442,-0.810940,1.352185,-0.674041,2.180966,0.530425,2.214392,-0.989151,0.386926,-0.352310
270,2,3,2,-0.001375,0.189071,NaN,-0.810709,NaN,-0.569412,-0.426171,0.220072,0.809200,0.912355
860,0,2,NaN,2.116151,-1.535981,0.279085,-2.348400,-1.119959,-1.069541,-0.478303,0.001006,0.745470,0.420702
435,2,2,2,-0.580361,-0.898720,-0.263709,-1.157469,-0.899476,-0.490832,-1.701312,-1.100063,-1.230007,-0.632837


In [ ]:
rf.bestX

,feature_10,feature_11,feature_12,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
29,2,1,2,-1.220715,-0.682635,0.102570,0.517777,-1.491929,1.662325,-0.691130,0.008394,-0.014518,0.428873
535,3,2,3,0.764024,-1.807970,-0.841080,0.035528,-0.028074,0.484714,1.496540,-0.632247,-0.014518,-1.073401
695,2,1,2,-0.360238,0.009083,-0.282353,-0.426767,-0.028074,-1.049076,-0.702097,0.008394,-1.648665,-0.027173
557,3,2,2,-0.047852,-0.211287,-0.267669,-0.065631,-0.028074,0.016807,-0.408133,-0.299202,-0.793206,0.212205
836,1,2,1,-1.640128,0.009083,0.015248,-1.336151,-0.028074,-1.051140,-0.065744,-0.723832,-1.419650,-0.081325
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,2,2,0.231442,-0.810940,1.352185,-0.674041,2.180966,0.530425,2.214392,-0.989151,0.386926,-0.352310
270,2,3,2,-0.001375,0.189071,0.015248,-0.810709,-0.028074,-0.569412,-0.426171,0.220072,0.809200,0.912355
860,0,2,2,2.116151,-1.535981,0.279085,-2.348400,-1.119959,-1.069541,-0.478303,0.001006,0.745470,0.420702
435,2,2,2,-0.580361,-0.898720,-0.263709,-1.157469,-0.899476,-0.490832,-1.701312,-1.100063,-1.230007,-0.632837


In [ ]:
# rf._find_best_hyperparameters()

In [ ]:
# best = rf.best_hyperparameters

In [ ]:
rf.X

# _missing_mask()

In [ ]:
rf.fit()

Fitting model...:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters for column feature_10: {'warm_start': True, 'n_jobs': -1, 'n_estimators': 1500, 'min_weight_fraction_leaf': 0.0, 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_impurity_decrease': 0.25, 'max_samples': 0.5, 'max_leaf_nodes': 10, 'max_features': 0.5, 'max_depth': 50, 'criterion': 'gini', 'class_weight': None, 'ccp_alpha': 0.0, 'bootstrap': True}


Fitting model...:   0%|          | 0/5 [05:21<?, ?it/s]


TypeError: bad operand type for unary ~: 'Categorical'

In [ ]:
rf.best_hyperparameters

{'feature_0': {'n_estimators': 500,
  'min_samples_split': 20,
  'min_samples_leaf': 1,
  'max_features': 'sqrt',
  'max_depth': 20,
  'bootstrap': False}}

In [ ]:
print(f"original data: X.shape: {X.shape}")
print(f"randomly masked data: Xm.shape: {Xm.shape}")
print(f"imputed (transformed) data: rf.transform(Xm).shape: {rf.transform(Xm).shape}")

original data: X.shape: (1000, 13)
randomly masked data: Xm.shape: (1000, 13)


AttributeError: 'IterativeRF' object has no attribute 'transform'

In [ ]:
rf.transform(Xm).shape

Column 0 shape: (10000,)
Column 1 shape: (10000,)
Column 2 shape: (10000,)
Column 3 shape: (10000,)
Column 4 shape: (10000,)
Column 5 shape: (10000,)
Column 6 shape: (10000,)
Column 7 shape: (10000,)
Column 8 shape: (10000,)
Column 9 shape: (10000,)


(10000, 10)

In [ ]:
X_test = rf.transform(Xm)
X_test

Column 0 shape: (10000,)
Column 1 shape: (10000,)
Column 2 shape: (10000,)
Column 3 shape: (10000,)
Column 4 shape: (10000,)
Column 5 shape: (10000,)
Column 6 shape: (10000,)
Column 7 shape: (10000,)
Column 8 shape: (10000,)
Column 9 shape: (10000,)


array([[ 1.57587581,  1.57587581,  1.57587581, ...,  1.57587581,
         1.57587581,  1.57587581],
       [ 0.89186302,  0.89186302,  0.89186302, ...,  0.89186302,
         0.89186302,  0.89186302],
       [        nan,         nan,  0.0268534 , ...,         nan,
                nan,         nan],
       ...,
       [-2.39030437, -2.39030437, -2.39030437, ..., -2.39030437,
        -2.39030437, -2.39030437],
       [ 0.91347373,  0.91347373,  0.91347373, ...,  0.91347373,
         0.91347373,  0.91347373],
       [-0.41222075, -0.41222075, -0.41222075, ..., -0.41222075,
        -0.41222075, -0.41222075]])

In [ ]:
np.sum(np.sum(np.power((X_test - X), 2)) / np.sum(missing_indicator))

ValueError: Unable to coerce to DataFrame, shape must be (1000, 10): given (10000, 10)

In [ ]:
np.sum(np.sum(np.power((Xm.fillna(Xm.median(axis=0)) - X), 2)) / np.sum(missing_indicator))

1.0228720359444132